In [1]:
#예제 2.1 토큰화 코드

# 띄어쓰기를 기준으로 토큰화

input_text = "나는 최근 파리 여행을 다녀왔다."
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리
str2idx = {word: i for i, word in enumerate(input_text_list)}
# 아이디 -> 토큰 딕셔너리
idx2str = {i: word for i, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다.']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다.': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다.'}
input_ids:  [0, 1, 2, 3, 4]


In [2]:
#예제 2.2 토큰 아이디에서 벡터로 변환
import torch
import torch.nn as nn

embedding_dim = 16 # 임베딩 차원

# 임베딩 레이어 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 임베딩 레이어에 토큰 아이디를 입력하여 임베딩 벡터를 얻음
input_embeddings = embed_layer(torch.tensor(input_ids))

print("input embeddings: ", input_embeddings)
print("length of input embeddings: ", len(input_embeddings))
print("input embeddings shape", input_embeddings.unsqueeze(0).shape)


input embeddings:  tensor([[ 0.7770, -0.2625, -1.0974, -0.0044,  0.2460,  1.1503,  1.1895, -0.6989,
         -0.8676,  1.3241, -0.2955,  0.1588,  0.4450,  1.3490,  1.2603,  0.5509],
        [-0.0305, -0.0896, -0.6064,  0.1746, -0.5936, -0.9227,  0.8282, -0.6571,
         -1.3590,  0.3900,  0.1317, -1.0009,  0.7452, -2.0369,  2.4439, -1.3522],
        [-1.6635, -0.5571, -0.7051, -0.3117, -0.2677, -1.1479,  0.0657,  0.8396,
          0.5488,  0.5365,  1.2385,  1.4215,  0.2737, -0.0441,  0.7447, -0.8945],
        [-0.3179, -1.5514,  0.6538, -0.7509,  0.2890, -0.0280,  0.4640, -1.3025,
          0.8520, -0.0179,  0.2255,  0.4587, -1.2693,  0.9345,  0.6715,  1.0444],
        [ 1.2534, -0.3619, -0.9062, -1.9034,  2.3410, -0.5002,  0.6875,  2.0066,
          0.5438,  1.3301,  1.3833, -1.2271,  1.3359, -1.3306, -1.0203,  0.3393]],
       grad_fn=<EmbeddingBackward0>)
length of input embeddings:  5
input embeddings shape torch.Size([1, 5, 16])


In [3]:
# 예제 2.3 절대적 위치 인코딩

import torch
import torch.nn as nn

embedding_dim = 16 # 임베딩 차원
max_position = 12 # 최대 토큰 수

embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)

token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings.unsqueeze(0)

input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

In [4]:
print("input ids: ", input_ids)
print("position ids: ", position_ids)

input ids:  [0, 1, 2, 3, 4]
position ids:  tensor([[0, 1, 2, 3, 4]])


In [5]:
# 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, embedding_dim)
weight_k = nn.Linear(embedding_dim, embedding_dim)
weight_v = nn.Linear(embedding_dim, embedding_dim)

querys = weight_q(input_embeddings) 
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

print("querys shape: ", querys.shape)
print("keys shape: ", keys.shape)
print("values shape: ", values.shape)


querys shape:  torch.Size([1, 5, 16])
keys shape:  torch.Size([1, 5, 16])
values shape:  torch.Size([1, 5, 16])


In [6]:
# 예제 2.5 스케일 점곱 방식의 어텐션

from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1)
    # 쿼리와 키의 내적을 구하고 스케일링
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    
    # 소프트맥스 함수를 통해 어텐션 가중치 계산
    weights = F.softmax(scores, dim=-1)

    # 가중치와 값의 곱으로 어텐션 값 계산한다. 입력과 동일한 차원을 가진다.
    outputs = weights @ values
    return outputs


In [7]:
# 예제 2.6 어텐션 연산의 입력과 출력

print("원본 입력 형태:", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 후 입력 형태:", after_attention_embeddings.shape)

원본 입력 형태: torch.Size([1, 5, 16])
어텐션 후 입력 형태: torch.Size([1, 5, 16])


In [8]:
# 2.7 어텐션 연산을 수행하는 AttentionHead 클래스
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        # 헤드 차원
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 선형 층
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        # 키 벡터 생성을 위한 선형 층
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        # 값 벡터 생성을 위한 선형 층
        self.weight_v = nn.Linear(token_embed_dim, head_dim)

    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values)
        )
        return outputs
    
attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

### 2.3.4. 멀티 헤드 어텐션

- 트랜스포머의 핵심 구성 요소 중 하나인 멀티 헤드 어텐션에 대해 알아보자.
- 멀티 헤드 어텐션은 어텐션을 병렬로 수행하여 어텐션의 품질을 높이는 방법이다.

```mermaid
graph TD
    q[Query]
    k[Key]
    v[Value]

    linear1[Linear]
    linear2[Linear]
    linear3[Linear]

    q --> linear1
    k --> linear2
    v --> linear3

    scale[Scale Dot Product * h]
    connect[Connect]
    final_linear[Final Linear]

    linear1 --> scale
    linear2 --> scale
    linear3 --> scale

    scale --> connect
    connect --> final_linear
```

In [9]:
# 멀티 헤드 어텐션 구현

import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, h_head, is_causal=False):
        super().__init__()
        self.n_head = h_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        
        B, T, C = querys.size()
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 1번째 선형 층 적용
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 2번째 선형 층 적용
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # 3번째 선형 층 적용

        attention = compute_attention(querys, keys, values, self.is_causal) # h번의 스케일 점곱 어텐션 수행
        output = attention.transpose(1, 2).reshape(B, T, C) # 어텐션 결과를 연결
        output = self.concat_linear(output) # 마지막 선형층 적용
        return output
    
n_head = 4
mh_attention = MultiHeadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

## 2.4. 정규화와 피드 포워드 층

정규화란, 딥러닝 모델에서 입력이 일정한 분포를 갖도록 만들어 학습이 안정적이고 빠르게 이루어지도록 하는 방법이다. 과거에는 배치 정규화(Batch Normalization)이 주로 사용되었지만, 트랜스포머 아키텍처에서는 특징 차원에서 정규화를 수행하는 층 정규화(Layer Normalization)를 사용한다.
어텐션 연산이 입력 단어 사이의 관계를 계산해 토큰 임베딩을 조정하는 역할을 한다면 전체 입력문장을 이해하는 연산은 완전 연결 층(Fully Connected Layer)이다. 이 층은 입력과 출력의 차원이 같은 행렬 곱셈을 수행하고, 이후에 활성화 함수를 통과시켜 비선형성을 추가한다. 이러한 층을 피드 포워드 층(Feed Forward Layer)이라고 한다.

### 층 정규화 이해하기

벡터 $x$를 정규화한 $norm_x$는 벡터 $x$에서 $x$의 평균을 빼고 $x$의 표준편차로 나눠 평균이 $0$이고 표준편차가 $1$인 분포로 만든다.
$$
\text{norm}_x = \frac{x - \mu}{\sigma}
$$

**배치 정규화(Batch Normalization)**

배치 정규화는 모델에 입력으로 들어가는 미니 배치 사이의 정규화를 수행한다. 자연어 처리에서는 배치 정규화를 사용하지 않는 이유는 자연어 처리에서 배치에 들어가는 입력의 길이가 다양하기 때문이다.

**층 정규화(Layer Normalization)**

층 정규화는 배치 정규화와 달리 각 토큰 임베딩의 평균과 표준편차를 구해 정규화를 수행한다. 이를 통해 입력의 분포를 안정화시킬 수 있다.
이려면 배치 정규화의 단점을 보완할 수 있다.

**층 정규화의 2가지 방식**

**사후 정규화(Post-LN)**
어텐션관 피드 포워드 층 이후에 층 정규화를 수행하는 방식이다.

**사전 정규화(Pre-LN)**
층 정규화를 수행한 후에 어텐션과 피드 포워드 층을 수행하는 방식이다.

In [10]:
# 2.9 층 정규화 (사전 정규화)

import torch.nn as nn

norm = nn.LayerNorm(embedding_dim)
norm_x = norm(after_attention_embeddings)
print("norm_x shape: ", norm_x.shape)

print(norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data) # 실제로 평균과 표준편차 확인

norm_x shape:  torch.Size([1, 5, 16])
tensor([[-1.8626e-09,  0.0000e+00,  0.0000e+00, -2.2352e-08, -2.9802e-08]]) tensor([[1.0327, 1.0326, 1.0326, 1.0328, 1.0327]])
